In [4]:
import cvxpy as cp
import numpy as np
import torch
import numpy as np

# assigns weights to data points, currently all 1's just to test other functions
def w(x, y, model):
    return torch.ones(len(y))

# samples minibatch
def get_batch(x, y, batch_size):
    inds = np.random.choice(range(len(y)), size=batch_size, replace=False)
    return x[inds], y[inds]
    
# performs an iteration of the iteratively reweighted least squares algorithm
def step(x, y, model, optimizer, w, batch_size):
    # arguments are self explanatory except w needs to be a callable function, i used a lambda below in the test
    #crit = torch.nn.MSELoss()
    optimizer.zero_grad()
    x, y = get_batch(x, y, batch_size)
    y_hat = model(x)
    weights = w(x, y, model)
    loss = torch.norm((y_hat - y))**2/batch_size
    #loss = crit(y, y_hat)
    loss.backward(retain_graph=True)
    optimizer.step()
    return loss

# gets parameters of model as array, used to check convergence later
def get_params(model):
    params=[]
    param_generator = model.parameters()
    for param in param_generator:
        nums = param.flatten()
        for num in nums:
            params.append(num.item())
    return np.array(params)

# training loop that stops when parameters dont change much, we can explore other convergence criteria
def train(x, y, model, optimizer, w, batch_size, eps):
    done = False
    params = get_params(model)
    losses=[]
    i=0
    while not done:
        i+=1
        #print(i)
        loss = step(x, y, model, optimizer, w, batch_size)
        losses.append(loss)
        new_params = get_params(model)
        if np.linalg.norm(params-new_params)<eps:
            done = True
        else:
            params = new_params
    return losses   

In [5]:
# generate LP constraints

d = 1 # number of equality constraints
m = 2 # dimension of z
np.random.seed(1)
s0 = np.random.randn(d)
s0 = np.maximum(s0, 0)
rand_z = np.random.randn(m)
A = np.random.randn(d, m) # equality constraint matrix
b = A @ rand_z + s0 # vector for equality constraints


epsilon = 1 # radius of wasserstein ball

# maximum suboptimality weight calculation method
# bad coding as I didn't pass epsilon, A or b as parameters but am using the global values from above...



def ms(x, y, model):
    sens = torch.zeros(len(x))
    y_hat = model(x).detach().numpy()
    for i in range(len(y_hat)):
        
        # solve decision problem to get z0
        z = cp.Variable(m)
        prob = cp.Problem(cp.Minimize(y_hat[i,:].T@z),
                [A@z == b, z >= 0])
        prob.solve()
        z0 = z.value
        #check if decision problem unbounded, infeasible, or has an optimal solution
        print("decision problem",prob.status)
        best_value = -np.inf
        for s in [-1,1]:
            for j in range(y_hat.shape[1]):
                z_s = cp.Variable(m)
                h = cp.Variable(1)
                prob = cp.Problem(cp.Maximize(y_hat[i,:].T@(z0 - z_s) + epsilon*h),
                         [A@z == b, z_s >= 0, z0 - z_s <= h, z_s - z0 <= h, s*(z0[j] - z_s[j]) == h ])
                prob.solve()
                #check if sensitivity problem unbounded, infeasible, or has an optimal solution
                print("sensitivity problem",prob.status)
                value = prob.value
                print("decision z", z0)
                print("sensitivity z", z_s.value)
                print("sensitivity h", h.value)
                print("y_hat", y_hat[i,:])
                print("s", s)
                print("j", j)
                print("this_sens_val", value)
                
                
                
                if value > best_value:
                    best_value = value
                
        sens[i] = best_value
        print("sensitivity", sens[i])
    return sens



# generate data

n=100 # number of obervations
p = 3 # dimension of context variable x or number of predictors
batch_size = 100
eps = 1e-2 # convergence criteria
sigma = .03


true_model = torch.nn.Sequential(
      torch.nn.Linear(in_features=p, out_features=m)
)
true_params = get_params(true_model)
x = torch.randn(n, p)
y = true_model(x) + sigma*torch.randn(n,m)

In [6]:
# train sensitivity model
model = torch.nn.Sequential(
      torch.nn.Linear(in_features=p, out_features=m)
)
start_params = get_params(model)
optimizer = torch.optim.Adam(model.parameters())
#yo = lambda x, c, batch_size: w(x, c, batch_size) # this is the callable function to calculate weights
losses = train(x, y, model, optimizer, ms, batch_size, eps)
end_params = get_params(model)
print("true params are", true_params)
print("found params are", end_params)
print("relative errror is", np.linalg.norm(end_params-true_params)/np.linalg.norm(true_params))

decision problem optimal
sensitivity problem unbounded
decision z [-1.99947487e-09  2.10728086e+00]
sensitivity z None
sensitivity h None
y_hat [-1.2737178  1.1735569]
s -1
j 0
this_sens_val inf
sensitivity problem unbounded
decision z [-1.99947487e-09  2.10728086e+00]
sensitivity z None
sensitivity h None
y_hat [-1.2737178  1.1735569]
s -1
j 1
this_sens_val inf
sensitivity problem optimal
decision z [-1.99947487e-09  2.10728086e+00]
sensitivity z [-1.77761010e-09  2.10728086e+00]
sensitivity h [-1.89635184e-10]
y_hat [-1.2737178  1.1735569]
s 1
j 0
this_sens_val -4.255689134424756e-10
sensitivity problem optimal
decision z [-1.99947487e-09  2.10728086e+00]
sensitivity z [ 2.10728086e+00 -2.92560756e-11]
sensitivity h [2.10728086]
y_hat [-1.2737178  1.1735569]
s 1
j 1
this_sens_val 7.264375964260809
sensitivity tensor(inf)
decision problem optimal
sensitivity problem unbounded
decision z [-2.78722785e-09  2.10728086e+00]
sensitivity z None
sensitivity h None
y_hat [0.19655249 0.2713697

TypeError: 'NoneType' object is not subscriptable